In [7]:
import pipeline
import importlib
importlib.reload(pipeline)

<module 'pipeline' from 'C:\\Google Drive\\Machine Learning\\ml-covid\\scripts\\pipeline.py'>

In [8]:
filepath = '../data/covid_df_protocol_3.pkl'
df_train, df_test = pipeline.read_and_process_data(filepath)
del filepath

Checking variables with highest percentage of NAs in training set:

In [9]:
df_train.apply(lambda x: x.isnull().sum()/len(x)).sort_values(ascending=False)[:20]

E4_International support                 0.035732
H4_Emergency investment in healthcare    0.020798
H5_Investment in vaccines                0.013500
H2_Testing policy                        0.009576
EconomicSupportIndex                     0.008859
E2_Debt/contract relief                  0.008058
E3_Fiscal measures                       0.007298
H3_Contact tracing                       0.007087
ContainmentHealthIndex                   0.007045
GovernmentResponseIndex                  0.007045
E1_Income support                        0.004472
C5_Close public transport                0.003755
C3_Cancel public events                  0.003712
C4_Restrictions on gatherings            0.003712
C7_Restrictions on internal movement     0.003712
StringencyIndex                          0.003670
H1_Public information campaigns          0.003670
C8_International travel controls         0.003670
C6_Stay at home requirements             0.003670
C2_Workplace closing                     0.003670


In [10]:
pipeline.sanity_check(df_train, df_test)

Success: Features match
Failure: Data is not clean yet, NAs remaining


In [11]:
df_train = df_train.dropna()
df_test = df_test.dropna()

#Since we will be predicting confirmed cases first, we remove the other outcomes (deaths)
df_train = df_train.drop(['Deaths'], axis=1)
df_test = df_test.drop(['Deaths'], axis=1)

#Next we choose a target to split the data on x and y
x_train, y_train = pipeline.divide_target_and_features(df_train, 'Confirmed Cases')
x_test, y_test = pipeline.divide_target_and_features(df_test, 'Confirmed Cases')

In [12]:
from sklearn import linear_model
import importlib
importlib.reload(pipeline)
ls = linear_model.Lasso(alpha=0.5)
rg = linear_model.Ridge(alpha=0.5)
lreg = linear_model.LinearRegression()
ev = {}
models = [(ls, 'Lasso'),
          (rg, 'Ridge'),
          (lreg, 'Linear Regression')]

for m in models:
    (model, name) = m
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print('{}\n{}\n'.format(name + ': Features with highest magnitude coefficients in absolute value'
                            , pipeline.get_most_relevant_features(x_train, model, 10)))
    ev[name] = pipeline.metrics(y_pred, y_test, x_train, y_train, model)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3738094750790.4736, tolerance: 2037045742.7499552
  positive)


Lasso: Features with highest magnitude coefficients in absolute value
                              Feature    Coefficient
174  Country_United States of America  129920.720298
18                    StringencyIndex  -65818.858240
88                    Country_Germany  -20129.767087
98                       Country_Iran  -16359.794784
60                     Country_Canada  -15078.445291
102                     Country_Italy   14954.770571
96                      Country_India   12721.652640
2                C2_Workplace closing   10635.622829
21               EconomicSupportIndex  -10622.613264
53                     Country_Brazil   -9529.164611

Bias: 335702388.42
Mean squared error: 2327817121.37
RSS: 9630179431110.29
Variance score: 0.63

Ridge: Features with highest magnitude coefficients in absolute value
                              Feature   Coefficient
174  Country_United States of America  35507.794510
64                      Country_China -34800.597381
88                    C

In [19]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.7691365902943914

In [41]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
pf = poly.fit_transform(x_train)